In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.6.0.jar -O /home/jovyan/postgresql-jdbc.jar

In [1]:
import scripts.update_data as upd

upd.main()

2025-05-16 15:25:52.285103 - Data update completed successfully


In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PostgreSQL-Integration")
    .config("spark.jars", "/home/jovyan/postgresql-jdbc.jar")
    .config("spark.driver.extraClassPath", "/home/jovyan/postgresql-jdbc.jar")
    .config("spark.executor.extraClassPath", "/home/jovyan/postgresql-jdbc.jar")
    .getOrCreate()
)

db_url = "jdbc:postgresql://postgres:5432/spark_db"

POSTGRES_USER = "spark_user"
POSTGRES_PASSWORD = "spark_password"

In [3]:
df_src = (
    spark.read.format("jdbc")
    .option("url", db_url)
    .option("dbtable", "mock")
    .options(user=POSTGRES_USER, password=POSTGRES_PASSWORD)
    .load()
)
df_src.show(5)

# 1. Dim_Customer
df_dim_customer = (
    spark.read.format("jdbc")
    .option("url", db_url)
    .option("dbtable", "dim_customer")
    .options(user=POSTGRES_USER, password=POSTGRES_PASSWORD)
    .load()
)

# 2. Dim_Seller
df_dim_seller = (
    spark.read.format("jdbc")
    .option("url", db_url)
    .option("dbtable", "dim_seller")
    .options(user=POSTGRES_USER, password=POSTGRES_PASSWORD)
    .load()
)

# 3. Dim_Product
df_dim_product = (
    spark.read.format("jdbc")
    .option("url", db_url)
    .option("dbtable", "dim_product")
    .options(user=POSTGRES_USER, password=POSTGRES_PASSWORD)
    .load()
)

# 4. Dim_Store
df_dim_store = (
    spark.read.format("jdbc")
    .option("url", db_url)
    .option("dbtable", "dim_store")
    .options(user=POSTGRES_USER, password=POSTGRES_PASSWORD)
    .load()
)

# 5. Dim_Supplier
df_dim_supplier = (
    spark.read.format("jdbc")
    .option("url", db_url)
    .option("dbtable", "dim_supplier")
    .options(user=POSTGRES_USER, password=POSTGRES_PASSWORD)
    .load()
)

# Fact Table Creation
df_fact_sales = (
    spark.read.format("jdbc")
    .option("url", db_url)
    .option("dbtable", "fact_sales")
    .options(user=POSTGRES_USER, password=POSTGRES_PASSWORD)
    .load()
)

# Show results
print("=== Dimension Tables ===")
print("Dim_Customer:")
df_dim_customer.show(5, truncate=False)
print("\nDim_Seller:")
df_dim_seller.show(5, truncate=False)
print("\nDim_Product:")
df_dim_product.show(5, truncate=False)
print("\nDim_Store:")
df_dim_store.show(5, truncate=False)
print("\nDim_Supplier:")
df_dim_supplier.show(5, truncate=False)

print("\n=== Fact Table ===")
print("Fact_Sales:")
df_fact_sales.show(5, truncate=False)

# Count records
print("\nRecord Counts:")
print(f"Dim_Customer: {df_dim_customer.count()}")
print(f"Dim_Seller: {df_dim_seller.count()}")
print(f"Dim_Product: {df_dim_product.count()}")
print(f"Dim_Store: {df_dim_store.count()}")
print(f"Dim_Supplier: {df_dim_supplier.count()}")
print(f"Fact_Sales: {df_fact_sales.count()}")

+---+-------------------+------------------+------------+--------------------+----------------+--------------------+-----------------+-----------------+------------------+-----------------+----------------+--------------------+--------------+------------------+------------+----------------+-------------+----------------+----------+----------------+--------------+---------------+-------------+----------------+----------+--------------+----------+-----------+-------------+------------+--------------------+------------+--------------+-------------+------------+-------------+----------------+--------------------+--------------+---------------+--------------------+-------------------+-------------+----------------+--------------------+--------------+----------------+------------------+----------------+
| id|customer_first_name|customer_last_name|customer_age|      customer_email|customer_country|customer_postal_code|customer_pet_type|customer_pet_name|customer_pet_breed|seller_first_name|se